In [1]:
! pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 84% 68.0M/80.9M [00:00<00:00, 92.0MB/s]
100% 80.9M/80.9M [00:00<00:00, 103MB/s] 


In [4]:
#Extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the dependencies


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#Printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# Loading the data form csv file through pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' ,encoding = 'ISO-8859-1')

In [9]:
#checking the number of rows and columns in datafframe
twitter_data.shape

(1599999, 6)

In [10]:
#printing the firts 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#naming the column and reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' ,names=column_names , encoding = 'ISO-8859-1')

In [12]:
#checking the number of rows and columns in datafframe
twitter_data.shape

(1600000, 6)

In [13]:
#printing the firts 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
#checking the distribution of the target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Convert 4 to 1

In [16]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [17]:
#checking the distribution of the target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative tweet

1 --> Positive tweet

**Stemming**

Stemming is the process of reducing a word to its keyword

Example: Actress, Actors, Acting = Act

In [18]:
port_stem = PorterStemmer()

In [19]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z)]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '. join(stemmed_content)

  return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
#separating the data(tweet) and the label(target)
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [25]:
 print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into the training data and testing data

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [28]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
print(Y.shape,Y_train.shape,Y_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [31]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [33]:
#Converting textual data to numeircal data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [34]:
print(X_train)

  (0, 445429)	0.44847027007225926
  (0, 236284)	0.4199633498272828
  (0, 109947)	0.37539644434066355
  (0, 186300)	0.5277617141490534
  (0, 356385)	0.35880495150482017
  (0, 439038)	0.27259730573434854
  (1, 161587)	1.0
  (2, 289892)	0.16786212258862687
  (2, 133121)	0.20288824511789302
  (2, 151613)	0.18803024655167525
  (2, 179100)	0.1618938997103493
  (2, 411296)	0.15169457857994498
  (2, 268082)	0.2412217109596309
  (2, 445796)	0.334845258855
  (2, 78359)	0.3128270664883473
  (2, 435871)	0.32964511005054153
  (2, 408541)	0.32105663567743975
  (2, 130199)	0.2907961969603714
  (2, 409446)	0.18708516908211442
  (2, 125213)	0.18921325818952875
  (2, 109947)	0.4591341545480502
  (3, 173441)	0.3746433622631123
  (3, 413693)	0.27088622722125455
  (3, 390684)	0.3940609080253374
  (3, 56778)	0.5200244739707928
  :	:
  (1279996, 392192)	0.22064467395143508
  (1279996, 436329)	0.27191495428417395
  (1279996, 319901)	0.21254694479232764
  (1279996, 239147)	0.22367820231371613
  (1279996, 29250

In [35]:
print(X_test)

  (0, 423222)	0.17914923706331665
  (0, 411296)	0.31430525104314233
  (0, 401019)	0.34908561962893286
  (0, 390406)	0.2198389416409358
  (0, 280472)	0.17824221835302806
  (0, 272380)	0.45354185564542554
  (0, 172388)	0.28072645573641997
  (0, 139010)	0.23688253829426661
  (0, 133174)	0.2552411671657934
  (0, 106692)	0.3655348480607625
  (0, 68200)	0.26798934494503485
  (0, 31336)	0.16246850710054328
  (0, 15183)	0.17192604061495495
  (1, 368101)	0.24596899424874577
  (1, 349958)	0.473926300641176
  (1, 258097)	0.2875148505675207
  (1, 218737)	0.40288012973779125
  (1, 146279)	0.5752609556592562
  (1, 15183)	0.21103671088898562
  (1, 6509)	0.30733412883307426
  (2, 402744)	0.4317517452958625
  (2, 258154)	0.25656484417867265
  (2, 184410)	0.589177584624408
  (2, 89992)	0.3633855979684962
  (2, 34584)	0.3791436697886803
  :	:
  (319994, 123997)	0.45300994760704544
  (319995, 447306)	0.321081796792186
  (319995, 423222)	0.2262982285321019
  (319995, 418463)	0.2381613168419694
  (319995, 3

**Training the Machine Learning Model (Logistic Regression Model)**

In [36]:
model = LogisticRegression(max_iter=1000)

In [37]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

Accuracy Score

In [39]:
#Accuracy Score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [41]:
print('Accuracy score is : ', training_data_accuracy)

Accuracy score is :  0.81032578125


In [42]:
#Accuracy Score on the testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [43]:
print('Accuracy score is : ', test_data_accuracy)

Accuracy score is :  0.7780125


Model accuracy = 77.8 %

Saving the trained model

In [44]:
import pickle

In [45]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using this saved model for future predictions

In [46]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [50]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] ==0):
  print('Negative tweet')

else:
  print('Positive tweet')

1
[1]
Positive tweet


In [51]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] ==0):
  print('Negative tweet')

else:
  print('Positive tweet')

0
[0]
Negative tweet
